In [71]:
import pandas as pd
import scipy.io
import mpld3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pymatreader
from sklearn import preprocessing as p

mpld3.enable_notebook()

ImportError: cannot import name 'OneToOneFeatureMixin' from 'sklearn.base' (c:\Users\santapaw\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py)

In [70]:
x = [1,2,3,4]

print(p.normalize(x))

NameError: name 'p' is not defined

In [2]:
def preprocessing(x):
    
    def get_condition(x):
        x_mean = np.mean(x)
        x_min = np.min(x)
        return np.mean([x_min, x_min])
    
    def find_first_seq(x):
        pos = 0
        condition = get_condition(x)
        for i in range(len(x)-1):
            if abs(x[i] - x[i+1]) >= np.mean(x[:i]):
                break
            else: 
                pos += 1
        return np.mean(x[:pos]), pos
    
    def preprocess(x):
        dev, c = find_first_seq(x)
        for i in range(len(x)):
            x[i] -= dev
        return x
    
    def remove_negative_values(x):
        for i in range(len(x)):
            if x[i] < 0:
                x[i] = 0
        return x
    
    def filter_first_seq(x):
        ""
    
    return remove_negative_values(preprocess(x))


def interpolate(x):
    y = np.cos(-x**2/9.0)
    return scipy.interpolate.interp1d(x, range(len(x)), kind='cubic')

In [46]:
def load_file(name) -> dict:
    data_path = os.getcwd() + '\data'
    data_f = {}
    data = pymatreader.read_mat(data_path + "\\" + name)
    signal = data['tissue'][0]
    T_s = data['info']['acq']["Ts"]
    return {'signal' : data['tissue'][0], 'time' : np.linspace(0, 0+(T_s*len(signal)),len(signal),endpoint=False), 'ts' : T_s}

In [47]:
def load_data() -> dict:
    data_path = os.getcwd() + '\data'
    data = {}
    for file in os.listdir(data_path):
        if ".mat" in file:
            data[file.replace('.mat','')] = load_file(file)
    return data

In [52]:
data = load_data()

In [50]:
print(load_data()['info']['acq']["Ts"])
print(load_data()['info']['tissue'])
print(load_data()['tissue'][0])

KeyError: 'info'

In [244]:
def visualize_all():
    data = load_data()
    for k, v in data.items():
        if k == "exp17_roivelke_inp_tis_111007":
            v = v - np.mean(v[:25])
            p = sns.lineplot(x=range(len(v)), y=v)
            plt.figure()
            plt.grid(False) 
            plt.title(k) 
            plt.xlabel("time")
            plt.ylabel("intesity")
        else: 
            v = preprocessing(v)
            #p = sns.lineplot(x=range(len(v)), y=v)
            p = plt.scatter(x=range(len(v)),y=v)
            plt.figure()
            plt.grid(False) 
            plt.title(k) 
            plt.xlabel("time")
            plt.ylabel("intesity")       

In [53]:
data = load_data()

In [56]:
data = data['exp13_aifnova_inp_tis_111017']['signal']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [247]:
def remove_negative_values(x):
    max_s = max(x)/10
    for i in range(len(x)):
        if x[i] < 0 or x[i] < max_s:
            x[i] = 0.1
    return x

def pos_min_max(x):
    x = x.tolist()
    max_ind = x.index(np.max(x))
    min_ind = x.index(np.min(x)) 
    if min_ind < max_ind:
        return min_ind, max_ind
    
def ma(x):
    max_ = np.argmax(x)
    min_ = np.argmin(x)
    arr = np.where(x==min_)
    #min_ = arr[0][len(arr[0])-1]
    x = x.tolist()
    max_ind = x.index(np.max(x))
    min_ind = x.index(np.min(x))
    return x[min_:max_], max_ind, min_ind

In [248]:
import paramnormal

data = load_data()
x = preprocessing(data['exp13_aifnova_inp_tis_111017'])

x_n = x[:20]

b = scipy.signal.resample(x_n, len(x_n) * 2)
s = remove_negative_values(np.append(b,x_n[20:]))

print(s)
signal, max, min = ma(s)
#print(min)

s_2 = []
signal, max, min = ma(s)
s2 = scipy.signal.resample(s[min:max], len(s[min:max]) * 2)
[s_2.append(i) for i in [s[:min], s_2, s[max:]]]

b_2 = scipy.signal.resample(s, len(s) * 2)
x = np.linspace(np.min(s_2),np.max(s_2))

params = paramnormal.lognormal.fit(s)
dist = paramnormal.lognormal.from_params(params)
plt.figure()
plt.plot(x, dist.pdf(x), color='black')
plt.scatter(x, dist.pdf(x), color='blue', marker="*")
plt.xlabel('time [s]')
plt.ylabel('Backscatter intensity')
plt.title('Log Normal Model')
plt.show()

TypeError: 'int' object is not callable